In [1]:
import requests
import json
import base64
import pandas as pd
import matplotlib as mt

#Requesting access to the spotify web API
client_id = '16fd6188f83242288e8af5e299bc66dd'
client_secret = '6abc3f8f1a714265a17ad6423066ec43'

auth_header_b64 = base64.b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
auth_header = auth_header_b64.decode('ascii')

req_header =  {'Authorization' : f'Basic {auth_header}'}
payload = {"grant_type": "client_credentials"}
url = "https://accounts.spotify.com/api/token"
resp = requests.post(url, headers = req_header, data = payload)
token = resp.json()

token['access_token']

#Defining header with access token
header = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token['access_token']}"
}

In [2]:
# Gets 10 ablbums from artist and return data frame with list of album ids and names
def albums(artist):
    oi = requests.get(f"https://api.spotify.com/v1/artists/{artist}/albums?limit=10", headers = header)
    oi = pd.DataFrame(oi.json()['items'])
    #oi = oi[oi['album_group'] == 'album'] #exclude single tracks
#   oi['artist_name'] = oi['artists'][0][0]['name'] #Fetching artist's name
    oi = oi[['id', 'name', 'release_date']]
    return oi

#Function to get all tracks from one album
def tracks_albums(album_id):
    varb = requests.get(f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=20", headers = header)
    raw = pd.DataFrame(varb.json()['items'])
    varb = raw[['track_number']]
    varb['track_id'] = raw['id']
    #Guardando o nome do album:
    varb['album_id'] = album_id
    return varb

#Função para pegar lista de musicas de cada album
def album_to_track(id_artista):
    album_dos_artistas = albums(id_artista)
    tracks = map(tracks_albums, album_dos_artistas['id'])
    oi = map(lambda dt : dt.merge(album_dos_artistas, left_on = 'album_id', right_on = 'id'), tracks)
    return pd.concat(oi).drop(columns = ['id'])

def album_to_final(id_album):
    musicas = tracks_albums(id_album)
    tracks = map(audio_analysis, musicas['track_id'])
    tracks = pd.DataFrame(tracks)
    oi = tracks.merge(musicas, left_on = "id", right_on = 'track_id').drop(columns = ['id'])
    return oi.drop(columns = ['analysis_url', 'track_href', 'uri'])    
    
#Getting global audio analysis for a single track
def audio_analysis(track):
    oi = requests.get(f"https://api.spotify.com/v1/audio-features/{track}", headers = header).json()
    return oi

#Function receives an artist and returns all of his albuns with global analysis for each track within each album.
def describing_tracks(artist):
    try:
        tracks_artists = album_to_track(artist)
        tracks = map(audio_analysis, tracks_artists['track_id'])
        tracks = pd.DataFrame(tracks)
        oi = tracks.merge(tracks_artists, left_on = "id", right_on = 'track_id').drop(columns = ['id'])
    except:
        return {}
    return oi.drop(columns = ['analysis_url', 'track_href', 'uri'])

#Define query
def query_genre(genero):
    oi = requests.get(f"https://api.spotify.com/v1/search?q=genre:{genero}&type=artist&limit=50", headers = header).json()
    oi = pd.DataFrame(oi['artists']['items'])
    return oi

#Get many artists from different genres
def many_artists(genre: str):
    artistas = []
    for i in genre:
        try:
            artistas = artistas + list(query_genre(i)['id'])
        except:
            continue
    artistas = list(dict.fromkeys(artistas)) #without repeated artists
    return artistas

#Definindo api para busca de info low-level
def low_level(track):
    return requests.get(f"https://api.spotify.com/v1/audio-analysis/{track}", headers = header).json()

In [14]:
#Análise 1: nacionalidade
arthur_loesser = album_to_final("0eT1ie2L88zuPrNmHVi3lJ")
jcm = map(album_to_final, ["3xzZ4Uo3uzUAiGNNjn7dKm", "1UQ57nPOWkCsDHbIv5HCUJ"])
jcm = pd.concat(jcm)
jcm['artista'] = "Joao Carlos Martins"
arthur_loesser['artista'] = "Arthur Loesser"
bach = pd.concat([jcm, arthur_loesser])
bach.to_csv("bach.csv")

/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [89]:
# Análise 2 timeline:
gm = album_to_final("7g5MR7T0Z21IVXBvePjRma")
palestrina = album_to_final("6tu14dYvtoshR8cOlT2JOZ")
bach = album_to_final("0eT1ie2L88zuPrNmHVi3lJ")
mozart = album_to_final("6AwQLN4vbnZWSLoh5F35MZ")
beethoven = album_to_final("6EbaN53vVILVI8BNgkXL4D")
alban_berg = album_to_final("001nmkTbxYzwHlB7kJGszX")
stravinsky = album_to_final("13GSfOTgzZjwp8JhmxgZr0")

gm['artista'] = "Guillaume de Machaut"
palestrina['artista'] = "Palestrina"
bach['artista'] = "Bach"
mozart['artista'] = "Mozart"
beethoven['artista'] = "Beethoven"
alban_berg['artista'] = "Alban Berg"
stravinsky['artista'] = "Stravinsky"

erudito = [gm, palestrina, bach, mozart, beethoven, alban_berg, stravinsky]
erudito = pd.concat(erudito)
erudito.reset_index()

key = [low_level(i) for i in list(erudito['track_id'])]

key = [key[i]['track']['key_confidence'] for i in range(len(key))]
erudito['key_confidence'] = key
erudito.to_csv("erudito.csv")

/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [95]:

erudito

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,track_number,track_id,album_id,artista,key_confidence
0,0.212,0.07530,9,-24.819,0,0.0422,0.965,0.000179,0.1690,0.0890,80.599,audio_features,299067,4,1,69HhRquABlUpRCPYVG4TOP,7g5MR7T0Z21IVXBvePjRma,Guillaume de Machaut,0.355
1,0.154,0.24400,9,-20.175,1,0.0369,0.904,0.131000,0.1630,0.0977,135.203,audio_features,611200,4,2,3GZIFKywZot38CBxfm83xC,7g5MR7T0Z21IVXBvePjRma,Guillaume de Machaut,0.531
2,0.190,0.15600,10,-24.101,0,0.0349,0.911,0.000002,0.1110,0.0938,82.303,audio_features,359600,4,3,2jxiaLVEE7ErVOOxqLGtrW,7g5MR7T0Z21IVXBvePjRma,Guillaume de Machaut,0.656
3,0.106,0.10500,4,-24.431,1,0.0434,0.944,0.000037,0.0978,0.0486,78.327,audio_features,309067,4,4,2Ldc1HYTKvwH9IvvLpFFDO,7g5MR7T0Z21IVXBvePjRma,Guillaume de Machaut,0.000
4,0.180,0.09820,5,-24.137,1,0.0362,0.970,0.006620,0.3380,0.0614,125.833,audio_features,174133,4,5,4D5tgJIdLdYFC4jSoOCP8l,7g5MR7T0Z21IVXBvePjRma,Guillaume de Machaut,0.256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,0.198,0.00711,6,-35.708,0,0.0407,0.975,0.126000,0.6930,0.0994,128.832,audio_features,203813,4,13,2U59mONiYrpaJU0EvPFUIy,13GSfOTgzZjwp8JhmxgZr0,Stravinsky,0.360
13,0.593,0.55400,1,-13.806,1,0.0597,0.946,0.916000,0.4390,0.1200,136.539,audio_features,88467,4,14,4KaYW238mxvvus5dpaYwP8,13GSfOTgzZjwp8JhmxgZr0,Stravinsky,0.718
14,0.525,0.14400,7,-19.070,0,0.1140,0.973,0.949000,0.1340,0.0503,122.515,audio_features,40373,4,15,4yTjqXYnuhGqzetNlV9QFh,13GSfOTgzZjwp8JhmxgZr0,Stravinsky,0.329
15,0.376,0.02290,0,-21.199,0,0.0362,0.922,0.416000,0.1540,0.1650,116.313,audio_features,223053,4,16,7hiNV3C12inhpSzXXGDyf5,13GSfOTgzZjwp8JhmxgZr0,Stravinsky,0.000


In [5]:
# Análise 3: Jazz
cool = album_to_final("0QWea2w5Y6pSoSWHuc7JMf")
cool['artista'] = "Miles D. - Cool"

hard = album_to_final("7cZ6oBx0SEUPDAoxJtxNDh")
hard['artista'] = "John C. - Hard "

jazz0 = pd.concat([cool, hard])

jazz = []
for i in list(jazz0['track_id']):
    j = pd.DataFrame(low_level(i)['sections'])
    j['track_id'] = i
    jazz.append(j)

jazz = pd.concat(jazz)
jazz = jazz0.merge(jazz, left_on = 'track_id', right_on= 'track_id')

jazz.to_csv("jazz.csv")

/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/pasoneto/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# como conseguimos representar som no computador
# o que nós podemos fazer com essas representações
# futuro